In [ ]:
pip install tensorflow

In [2]:
import numpy as np
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
# Define constants
image_size = (48, 48)
batch_size = 32
num_classes = 5
epochs = 10

In [4]:
# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    'faceData/train',  # Path to training dataset
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 23845 images belonging to 5 classes.


In [5]:
# Model Architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

C:\Users\rsree\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
model.fit(train_generator, epochs=epochs)

Epoch 1/10


C:\Users\rsree\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


746/746 ━━━━━━━━━━━━━━━━━━━━ 91s 115ms/step - accuracy: 0.3181 - loss: 1.5512
Epoch 2/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 86s 115ms/step - accuracy: 0.4360 - loss: 1.3576
Epoch 3/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 85s 114ms/step - accuracy: 0.4932 - loss: 1.2479
Epoch 4/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 88s 117ms/step - accuracy: 0.5132 - loss: 1.2134
Epoch 5/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 87s 116ms/step - accuracy: 0.5340 - loss: 1.1593
Epoch 6/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 87s 115ms/step - accuracy: 0.5413 - loss: 1.1440
Epoch 7/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 87s 116ms/step - accuracy: 0.5489 - loss: 1.1144
Epoch 8/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 85s 112ms/step - accuracy: 0.5722 - loss: 1.0775
Epoch 9/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 87s 116ms/step - accuracy: 0.5757 - loss: 1.0648
Epoch 10/10
746/746 ━━━━━━━━━━━━━━━━━━━━ 87s 116ms/step - accuracy: 0.5837 - loss: 1.0418


In [8]:
# Save the trained model
model.save('facial_emotion_model.h5')

In [9]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('facial_emotion_model.h5')

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to perform emotion recognition
def predict_emotion(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        # Crop the face region
        face_roi = frame[y:y+h, x:x+w]
        
        # Resize and preprocess the face region
        face_roi = cv2.resize(face_roi, (48, 48))
        face_roi = face_roi.reshape((1, 48, 48, 3)) / 255.0
        
        # Perform prediction
        prediction = model.predict(face_roi)
        
        # Get the predicted emotion label
        emotion_label = ['angry', 'fear', 'happy', 'neutral', 'sad'][np.argmax(prediction)]
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Display the emotion label on the frame
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    
    return frame

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    # Perform emotion recognition
    frame_with_emotion = predict_emotion(frame)
    
    # Display the frame with emotion recognition
    cv2.imshow('Emotion Recognition', frame_with_emotion)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━